In [38]:
import numpy as np
np.random.seed(42)

In [39]:
import tensorflow as tf
import pdb
import fxns
reload(fxns)

<module 'fxns' from '/Users/fultonw/Documents/nonscratch/20180216_motifnet/motifnet/code/fxns.py'>

In [51]:
# constants
data_path = '../data/shuffled_examples'
out_path = '../output/shuffled_examples'
train_out_path = '%s/texttrain' % out_path
valid_out_path = '%s/textvalidate' % out_path
test_out_path = '%s/texttest' % out_path
test_chrs = [22]
valid_chrs = [21]
blocksize = 10
num_blocks = 4
create_data = True
text = False
binary = True
batch_size = 5
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

In [52]:
# create data
if create_data:
    fxns.create_shards(data_path,
            train_out_path, valid_out_path, test_out_path,
            test_chrs, valid_chrs,
            blocksize=blocksize,
            text=text, binary=binary, num_blocks=num_blocks)

> /Users/fultonw/Documents/nonscratch/20180216_motifnet/motifnet/code/fxns.py(44)create_shards()
-> def file_input(files):
(Pdb) c
> /Users/fultonw/Documents/nonscratch/20180216_motifnet/motifnet/code/fxns.py(65)create_shards()
-> train_readers = {chr:open(train_chr_to_files[chr]) for chr in train_chr_to_files.keys()}
(Pdb) c
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5


In [53]:
# get training data
tf.reset_default_graph()
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label], batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv], batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

In [62]:
# print data
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    _seq_batch, _label_batch = sess.run([seq_batch, label_batch])
#     print _seq_batch, _label_batch
    print _seq_batch.shape, _label_batch.shape
    print sess.run(tf.reshape(seq_batch, (-1, 4000/4,4))).sum(axis=0)
    coord.request_stop()
    coord.join(threads)

(5, 4000) (5, 24)
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [55]:
# define model
loss = fxns.get_logreg_model(seq_batch, label_batch, info['seq_len'], info['label_len'])
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [56]:
# train

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(1000):
        _, _loss, a, b = sess.run([optimizer, loss, seq_batch, label_batch])
        
        if i % 100 == 0:
            av, bv = sess.run([seqv_batch,labelv_batch])
            _lossv = sess.run(loss,
                feed_dict={
                    seq_batch:av,
                    label_batch:bv
                })
            print i, _loss, _lossv
    
    coord.request_stop()
    coord.join(threads)

0 0.6903673 0.64162725
100 0.1838389 0.20634474
200 0.12751153 0.17259066
300 0.108013585 0.16089983
400 0.09999327 0.16075632
500 0.07770193 0.16465083
600 0.06950669 0.18527451
700 0.05380475 0.16772635
800 0.049308818 0.16274077
900 0.04449594 0.18214272
